# Tomographic indexing notebook with minor phase adjustments  
__Written by Haixing Fang, Jon Wright and James Ball__  
__Date: 21/02/2025__

This notebook will try to find good grain orientations from a 4D merge of your peak data.  
This notebook is optimised for a weaker minor phase in a larger major phase sample.  
This means your peaks have been merged across omega and dty.  
This notebook (and the tomo route in general) works best for low levels of deformation.  
If it doesn't seem to work well, try the point-by-point route instead!

In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())

In [ ]:
# this cell is tagged with 'parameters'
# to view the tag, select the cell, then find the settings gear icon (right or left sidebar) and look for Cell Tags

# python environment stuff
PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

# dataset file to import
dset_path = 'si_cube_test/processed/Si_cube/Si_cube_S3DXRD_nt_moves_dty/Si_cube_S3DXRD_nt_moves_dty_dataset.h5'

# phase names
major_phase_strs = ['Fe']
minor_phase_str = 'Au'

# peak filtering
min_frames_per_peak = 0
major_phase_cf_frac = 0.99418
major_phase_cf_dstol = 0.005

minor_phase_cf_frac = 0.9975
minor_phase_cf_dsmax = 1.594
minor_phase_cf_dstol = 0.005

# indexing
rings_for_gen = [0, 4, 5]

# now we want to decide which rings to score our found orientations against
# generally we can just exclude dodgy rings (close to other phases, only a few peaks in etc)
rings_for_scoring = [0, 2, 3, 4, 5, 6, 7, 8, 10, 12, 13]

# the sequence of hkl tolerances the indexer will iterate through
hkl_tols_seq = [0.01, 0.02, 0.03, 0.04, 0.05]
# the sequence of minpks fractions the indexer will iterate through
fracs = [0.9]
# the tolerance in g-vector angle

# the max number of UBIs we can find per pair of rings
max_grains = 1000

peak_assign_tol = 0.025

In [ ]:
# import functions we need

import numpy as np
import h5py

import matplotlib
%matplotlib ipympl
from matplotlib import pyplot as plt

import ImageD11.nbGui.nb_utils as utils

import ImageD11.grain
import ImageD11.indexing
import ImageD11.columnfile
from ImageD11.peakselect import select_ring_peaks_by_intensity, remove_peaks_from_phases

# Load data

## Dataset

In [ ]:
ds = ImageD11.sinograms.dataset.load(dset_path)
print(ds)

## Phases
If the parameter file was a json, we can access the unit cells via `ds.phases.unitcells`

In [1]:
ds.phases = ds.get_phases_from_disk()
ds.phases.unitcells

NameError: name 'ds' is not defined

Now we need to decide which are our 'major' phases, which we will remove from our peaks before indexing.

In [ ]:
major_phase_unitcells = [ds.phases.unitcells[mps] for mps in major_phase_strs]
print(*major_phase_unitcells)
minor_phase_unitcell = ds.phases.unitcells[minor_phase_str]
print(minor_phase_unitcell)

## Peaks

In [ ]:
cf_4d = ds.get_cf_4d()
ds.update_colfile_pars(cf_4d)  # computes geometry, needed for filtration

In [ ]:
# Optionally remove some noisy peaks
if min_frames_per_peak > 0:
    cf_4d.filter(cf_4d['npk2d'] > min_frames_per_peak)

# Filtration
## Remove major phase peaks

In [ ]:
cf_strong = remove_peaks_from_phases(cf_4d, major_phase_cf_dstol, major_phase_unitcells)
ds.update_colfile_pars(cf_strong, phase_name=minor_phase_str)

# Filtration
Now we are filtering our peaks (`cf_4d`) to select only the strongest ones for indexing purposes only!  
We first filter the peaks in $d^{*}$ to keep only those close to the predicted peaks from the unit cell.  
We then sort our peaks by intensity, and take a certain intensity-weighted fraction of them.  
`dstol`: The tolerance in $d^{*}$ between a peak and a predicted reflection.  
`dsmax`: The maximum allowed peak $d^{*}$ value. Used to limit the number of rings given to the indexer - 6-8 rings max are normally sufficient.   
`frac`: The intensity fraction: `frac=0.9` keeps 90% of the peak intensity. We recommend that you choose a value close to the 'elbow' of the plot.

In [ ]:
cf_strong = select_ring_peaks_by_intensity(cf_strong, frac=minor_phase_cf_frac, dsmax=minor_phase_cf_dsmax, dstol=minor_phase_cf_dstol, doplot=0.5)

In [ ]:
skip = 1  # we can skip peaks to speed up plotting if needed
fig, ax = plt.subplots(figsize=(16, 9), constrained_layout=True)
ax.plot(cf_4d.ds[::skip], cf_4d.sum_intensity[::skip],',', label='cf_4d',c='blue')
ax.plot(cf_strong.ds[::skip], cf_strong.sum_intensity[::skip],',', label='minor phase',c='green')
ax.vlines(minor_phase_unitcell.ringds, 5e3, 1e4, zorder=0, color='red')
ax.set(xlabel=r'$d^{*}~(\AA^{-1})$', ylabel='Intensity', yscale='log', title='Peak filtration')
ax.legend()
plt.show()

# Indexing
## Ring assignment

In [ ]:
indexer = ImageD11.indexing.indexer_from_colfile_and_ucell(cf_strong, minor_phase_unitcell)
indexer.ds_tol = minor_phase_cf_dstol
ImageD11.indexing.loglevel = 1
indexer.assigntorings()
ImageD11.indexing.loglevel = 3
print(f"Indexing {cf_strong.nrows} peaks")

In [ ]:
skip = 1  # we can skip peaks to speed up plotting if needed
fig, ax = plt.subplots(layout='constrained', figsize=(10,5))
ax.scatter(indexer.colfile.ds[::skip], indexer.colfile.eta[::skip], c=indexer.ra[::skip]%20, cmap='tab20', s=1)
ax.vlines(minor_phase_unitcell.ringds, -50, 50, zorder=0, color='red')
ax.set(xlabel=r'$d^{*}~(\AA^{-1})$', ylabel=r'$\eta~(\degree)$', xlim=(min(ucell.ringds[0], cf_strong.ds.min()) - 0.02, cf_strong.ds.max() + 0.02), title='Ring assignments')
plt.show()

In [ ]:
cosine_tol = np.cos(np.radians(90 - ds.ostep))
grains, indexer = utils.do_index(cf=cf_strong,
                                 unitcell=minor_phase_unitcell,
                                dstol=minor_phase_cf_dstol,
                                forgen=rings_for_gen,
                                foridx=rings_for_scoring,
                                hkl_tols=hkl_tols_seq,
                                fracs=fracs,
                                cosine_tol=cosine_tol,
                                max_grains=max_grains
)
for ginc, g in enumerate(grains):
    g.gid = ginc
print(f'Found {len(grains)} grains!')

# Results

In [ ]:
unit_cell_lengths = np.array([np.sort(g.unitcell.copy()[:3]) for g in grains])
median_unit_cell = np.median(unit_cell_lengths, axis=0)

fig, ax = plt.subplots(constrained_layout=True)
ax.plot(unit_cell_lengths)
ax.hlines(median_unit_cell, 0, len(unit_cell_lengths))
ax.set(xlabel='Grain ID', ylabel='Unit cell length')
plt.show()

In [ ]:
utils.assign_peaks_to_grains(grains, cf_strong, tol=peak_assign_tol)

In [ ]:
utils.plot_index_results(indexer, cf_strong, 'Indexing attempt')

In [ ]:
utils.plot_grain_sinograms(grains, cf_strong, min(len(grains), 25))

# Export data

In [ ]:
ds.save_grains_to_disk(grains, phase_name=minor_phase_str)

In [ ]:
ds.save()